# 1. 環境安裝

In [ ]:
# 1. 安裝環境
!pip install -q datasets transformers torch accelerate setfit scikit-learn matplotlib seaborn
!pip install hf_xet
!pip install openpyxl

# 2. 掛載 Google Drive
from google.colab import drive
import os
import shutil

drive.mount('/content/drive')

# 設定你的檔案在 Drive 中的路徑
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Project/Sarcasm-Enhanced-SetFit/imdb_analysis_results"
CSV_PATH = os.path.join(DRIVE_PATH, "qwen_labeled_data.csv")

# 檢查一下檔案在不在
if os.path.exists(DRIVE_PATH):
    print(f"✅ 成功連接 Drive，檔案列表: {os.listdir(DRIVE_PATH)}")
else:
    print("❌ 找不到路徑，請確認你的資料夾名稱是否正確")


DRIVE_SAVE_PATH = "/content/drive/MyDrive/Colab Notebooks/Project/Sarcasm-Enhanced-SetFit/Model_colab"
print(f"📂 模型將儲存於雲端路徑: {DRIVE_SAVE_PATH}")

In [ ]:
# 各項基本設定
USER_CONFIG = {
    "data": {
        "dataset_name": "imdb",
        "split": "train",
        "seed_count": 800,  # 練習用種子資料數量
        "random_seed": 42
    },
    "llm": {
        "model_id": "Qwen/Qwen2.5-1.5B-Instruct",
        "temperature": 0.1,  # 降低隨機性，保證標註一致性
        "max_new_tokens": 100
    },
    "labels": {
        "aspects": ["Acting", "Plot/Story", "Visuals/Effects", "Pacing"],
        "sarcasm": ["Yes", "No"]
    }
}

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# ==========================================
# 視覺化：酸民比例統計圖
# ==========================================

def plot_sarcasm_statistics(df):
    print("正在計算各分類的酸民滲透率...")

    order_list = df['aspect'].value_counts().index
    # order_list = ['Plot/Story', 'Acting', 'Visuals/Effects', 'Pacing']

    # 1. 計算每個 Aspect 中，Sarcasm Yes/No 的數量
    # 使用 crosstab 交叉分析
    sarcasm_counts = pd.crosstab(df['aspect'], df['sarcasm'])
    sarcasm_counts = sarcasm_counts.reindex(order_list)

    # 2. 轉換成比例 (Percentage) 以便公平比較
    # (Plot 總數多，光看數量不準，要看比例)
    sarcasm_ratio = sarcasm_counts.div(sarcasm_counts.sum(1), axis=0) * 100

    # 3. 畫圖
    custom_palette = {"No": "#4A90E2", "Yes": "#FF6B6B"}
    fig, ax = plt.subplots(1, 2, figsize=(20, 8))

    # 左圖：絕對數量 (Count)
    sns.countplot(data=df, x='aspect', hue='sarcasm',
                  palette=custom_palette, ax=ax[0], order=order_list)
    ax[0].set_title("Absolute Count of Sarcasm per Aspect", fontsize=14, fontweight='bold')
    ax[0].set_xlabel("Aspect", fontsize=12)
    ax[0].set_ylabel("Number of Reviews", fontsize=12)
    ax[0].legend(title='Sarcasm', title_fontsize='10')

    for container in ax[0].containers:
      ax[0].bar_label(container, padding=3, fontsize=11, fontweight='bold')

    # 右圖：滲透比例 (Percentage)
    sarcasm_ratio.plot(kind='bar', stacked=True,
                       color=custom_palette, ax=ax[1], alpha=0.9, rot=0)
    ax[1].set_title("Sarcasm Penetration Rate (%) by Aspect", fontsize=14, fontweight='bold')
    ax[1].set_ylabel("Percentage (%)")
    ax[1].legend(title='Sarcasm', loc='upper right')

    # 在右圖的柱子上標示數字
    for c in ax[1].containers:
        labels = [f'{v.get_height():.1f}%' if v.get_height() > 0 else '' for v in c]
        ax[1].bar_label(c, labels=labels, label_type='center', color='white', fontweight='bold')

    sns.despine(left=True)
    plt.tight_layout()
    plt.show()

    filename = "Sarcasm Analysis Bar.png"
    save_path = os.path.join(OUTPUT_DIR, filename)
    plt.savefig(save_path, dpi=300)
    print(f"✅ 圖表已儲存: {save_path}")

# 執行統計

if os.path.exists(CSV_PATH):
    labeled_df = pd.read_csv(CSV_PATH)
else:
    raise ValueError("❌ 找不到 qwen_labeled_data.csv，請確認檔案位置。")

if 'labeled_df' in locals():
    plot_sarcasm_statistics(labeled_df)

# 2. 視覺化與分群分析 - aspect_model

## 載入訓練完成的A、B模型

In [ ]:
import os
from setfit import SetFitModel
from google.colab import drive
import torch

# ==========================================
# 1. 掛載雲端硬碟
# ==========================================
# 如果已經掛載過，這行會自動略過；如果還沒，會跳出授權視窗
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# ==========================================
# 2. 設定模型路徑
# ==========================================
BASE_PATH = DRIVE_SAVE_PATH

# 定義兩個模型的資料夾名稱 (根據您的指示)
MODEL_A_NAME = "IMDB_aspect_model"          # Baseline (原始模型)
MODEL_B_NAME = "IMDB_aspect_model_enhanced" # Enhanced (反諷增強模型)

path_a = os.path.join(BASE_PATH, MODEL_A_NAME)
path_b = os.path.join(BASE_PATH, MODEL_B_NAME)

# 檢查一下路徑是否存在，避免讀取失敗
print(f"📂 檢查路徑: {BASE_PATH}")
if os.path.exists(BASE_PATH):
    print("   目錄內容:", os.listdir(BASE_PATH))
else:
    print("❌ 找不到 Base Path，請確認路徑是否正確！")

# ==========================================
# 3. 載入模型
# ==========================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\n🚀 使用裝置: {device}")

try:
    # 載入 Model A
    if os.path.exists(path_a):
        print(f"   正在載入 Model A: {MODEL_A_NAME}...")
        model_a = SetFitModel.from_pretrained(path_a).to(device)
        print("   ✅ Model A 載入成功！")
    else:
        print(f"   ❌ 找不到 Model A 資料夾: {path_a}")

    # 載入 Model B
    if os.path.exists(path_b):
        print(f"   正在載入 Model B: {MODEL_B_NAME}...")
        model_b = SetFitModel.from_pretrained(path_b).to(device)
        print("   ✅ Model B 載入成功！")
    else:
        print(f"   ❌ 找不到 Model B 資料夾: {path_b}")

except Exception as e:
    print(f"\n❌ 發生錯誤: {e}")

## 繪製混淆矩陣

In [ ]:
def plot_beautiful_confusion_matrix(cm, labels, title, figsize=(10, 8), cmap='YlGnBu', show_percent=True):
    """
    繪製美觀、易讀的混淆矩陣 (使用 Seaborn)

    Args:
        cm (numpy array): 混淆矩陣數值
        labels (list): 標籤名稱列表
        title (str): 圖表標題
        figsize (tuple): 圖片大小
        cmap (str): Seaborn/Matplotlib 色票名稱 (推薦: 'YlGnBu', 'viridis', 'Blues', 'rocket_r')
        show_percent (bool): 是否在格子中顯示百分比 (Row-wise)
    """

    # 1. 計算百分比矩陣 (Row-wise: 每一列總和為 100%)
    # 加上 1e-10 是為了避免除以零的錯誤
    cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

    # 2. 準備格子裡的註釋文字 (Annot)
    if show_percent:
        annot_labels = [
            [f"{value}\n({percent:.1f}%)" for value, percent in zip(row_cm, row_perc)]
            for row_cm, row_perc in zip(cm, cm_percent)
        ]
    else:
        annot_labels = cm # 只顯示數字

    # 3. 開始繪圖
    fig, ax = plt.subplots(figsize=figsize)

    sns.heatmap(cm, annot=annot_labels, fmt='', cmap=cmap, ax=ax,
                xticklabels=labels, yticklabels=labels,
                linewidths=1, linecolor='lightgray',
                annot_kws={"size": 12, "weight": "bold"}, # 設定格子內文字大小和粗細
                cbar_kws={'shrink': .8} # 稍微縮小 colorbar
               )

    # 4. 美化標題與座標軸
    ax.set_title(title, fontsize=18, fontweight='bold', pad=20)
    ax.set_xlabel('Predicted Label', fontsize=14, labelpad=10)
    ax.set_ylabel('True Label', fontsize=14, labelpad=10)

    # 調整座標軸刻度文字的大小
    plt.xticks(fontsize=11, rotation=0) # 如果標籤太長可以改 rotation=45
    plt.yticks(fontsize=11, rotation=0)

    filename = title.replace(":", "-") + ".png"
    save_path = os.path.join(DRIVE_SAVE, filename)
    plt.savefig(save_path, dpi=300)
    print(f"✅ 圖表已儲存: {save_path}")

    plt.tight_layout()
    plt.show()

    return fig



def run_visualization_analysis(model, dataset, label_map, model_name):
    print(f"Generating predictions and embeddings with {model_name}...")

    # 1. 取得預測結果與 Embeddings
    # SetFit 的 model_body 就是底層的 Sentence Transformer
    embeddings = model.model_body.encode(dataset['text'])
    predictions = model.predict(dataset['text'])

    # 將 label id 轉回文字 (0 -> Acting)
    id2label = {v: k for k, v in label_map.items()}
    true_labels = [id2label[x] for x in dataset['label']]
    pred_labels = [id2label[x] for x in predictions.tolist()]

    # 2. 混淆矩陣 (Confusion Matrix)
    print("\n--- Model Performance ---")
    cm = confusion_matrix(true_labels, pred_labels, labels=list(label_map.keys()))

    plot_beautiful_confusion_matrix(
        cm,
        label_map.keys(),
        title=f'Confusion Matrix : True vs Predicted ({model_name})',
        figsize=(12, 8),
        cmap='Blues',
        show_percent=True
        )

    # 3. t-SNE 降維視覺化 (Embeddings Space)
    print("\n--- Embedding Space Visualization (t-SNE) ---")
    # 參數設定：perplexity 需小於樣本數，通常設 30，但我們資料少，設 10-20 較穩
    tsne = TSNE(n_components=2, random_state=42, perplexity=10, init='pca', learning_rate='auto')
    vis_dims = tsne.fit_transform(embeddings)

    vis_df = pd.DataFrame({
        'x': vis_dims[:, 0],
        'y': vis_dims[:, 1],
        'Aspect': true_labels,
        'Predicted': pred_labels
    })

    # 繪製真實標籤分佈
    plt.figure(figsize=(12, 8))
    sns.scatterplot(data=vis_df, x='x', y='y', hue='Aspect', style='Aspect', s=100, palette='viridis')
    plt.title('t-SNE Visualization of Movie Reviews (Colored by Aspect)')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.tight_layout()
    plt.show()

    # 4. K-Means 非監督分群 (Unsupervised Clustering)
    # 強迫機器把資料分成 4 堆，看它怎麼分
    print("\n--- K-Means Clustering Analysis ---")
    kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(embeddings)

    plt.figure(figsize=(12, 8))
    sns.scatterplot(data=vis_df, x='x', y='y', hue=cluster_labels, palette='tab10', s=100, legend='full')
    plt.title('K-Means Clustering Results (Machine\'s own logic)')
    plt.show()



def run_sarcasm_visualization(model_b, dataset, label_map, model_name="Model B (Enhanced)"):
    print("   (直接使用 Ground Truth 反諷標籤注入特徵)")

    # 1. 轉成 Pandas 方便處理
    # 注意：這裡假設 dataset 有 'sarcasm' 這個欄位
    # 如果 dataset 是從 labeled_df 轉來的，應該會有。如果沒有，我們要從原始 dataframe 撈。
    if isinstance(dataset, Dataset):
        df_temp = dataset.to_pandas()
    else:
        df_temp = dataset.copy()

    # 檢查有沒有 sarcasm 欄位
    if 'sarcasm' not in df_temp.columns:
        print("❌ 錯誤：資料集中找不到 'sarcasm' 欄位，無法進行 sarcasm 測試。")
        return

    # 2. 加工文字 (注入標籤)
    print("   Step 1: 根據真實標籤注入 [Sarcasm]...")

    def inject_ground_truth(row):
        if row['sarcasm'] == 'Yes':
            return f"[Sarcasm] {row['text']}"
        else:
            return row['text']

    df_temp['text'] = df_temp.apply(inject_ground_truth, axis=1)

    # 統計一下
    sarcasm_count = len(df_temp[df_temp['text'].str.startswith("[Sarcasm]")])
    print(f"   (共 {sarcasm_count} 筆資料被強制標記為反諷)")

    # 3. 轉回 Dataset
    dataset_for_model_b = Dataset.from_pandas(df_temp)

    # 4. 呼叫原本的繪圖函式
    print(f"   Step 2: 開始繪圖...")
    run_visualization_analysis(model_b, dataset_for_model_b, label_map, model_name)

In [ ]:
label_mapping = {
    'Acting': 0,
    'Plot/Story': 1,
    'Visuals/Effects': 2,
    'Pacing': 3
}

# 1. 準備包含 'sarcasm' 欄位的完整資料集
# 我們直接用 labeled_df 來做轉換，確保欄位都在
full_dataset_df = labeled_df[labeled_df['aspect'].isin(USER_CONFIG['labels']['aspects'])].copy()
full_dataset_df['label'] = full_dataset_df['aspect'].map(label_mapping)
full_ds_for_vis = Dataset.from_pandas(full_dataset_df)

# 2. 執行 Model A (對照組 - 原始)
print("\n🔵 [Model A] 原始模型分析...")
aspect_model = model_a
run_visualization_analysis(aspect_model, full_ds_for_vis, label_mapping, "Model A (Baseline)")

# 3. 執行 Model B (實驗組 - 反諷標記)
print("\n🔴 [Model B] 加入反諷標記模型...")
aspect_model_enhanced = model_b
run_sarcasm_visualization(target_model, full_ds_for_vis, label_mapping, "Model B (Enhanced)")

# 3. 視覺化與分群分析 - sentiment_model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
from setfit import SetFitModel
import os
import torch
from google.colab import drive

# ==========================================
# 1. 設定路徑與掛載
# ==========================================
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 設定專案路徑 (請確認這是存放模型的地方)
OUTPUT_DIR = DRIVE_SAVE_PATH

# 模型路徑
PATH_A = os.path.join(OUTPUT_DIR, "Sentiment_Model_A_Baseline")
PATH_B = os.path.join(OUTPUT_DIR, "Sentiment_Model_B_Enhanced")

# ==========================================
# 2. 抓出「未經訓練」的 200 筆測試資料
# ==========================================
print("📦 正在準備測試資料 (Test Set Isolation)...")

if os.path.exists(CSV_PATH):
    df = pd.read_csv(CSV_PATH)
else:
    backup_csv = os.path.join(OUTPUT_DIR, "qwen_labeled_data.csv")
    if os.path.exists(backup_csv):
        df = pd.read_csv(backup_csv)
    else:
        raise ValueError("❌ 找不到 qwen_labeled_data.csv，請確認檔案位置。")

# A. 重現訓練時的篩選與抽樣邏輯
# 抽出訓練使用的兩種資料各300筆，總共排除600筆，剩下的200筆用於測試
df_clean = df[df['label_original'].isin([0, 1])].copy()

# 訓練時每類取 300 筆，random_state=42
SAMPLES_PER_CLASS = 300
train_df_indices = df_clean.groupby('label_original', group_keys=False).apply(
    lambda x: x.sample(min(len(x), SAMPLES_PER_CLASS), random_state=42)
).index

print(f"   - 原始有效資料: {len(df_clean)} 筆")
print(f"   - 訓練集使用: {len(train_df_indices)} 筆")

# B. 使用 Drop 排除訓練資料 -> 剩下的就是測試集
test_df = df_clean.drop(train_df_indices).copy()

print(f"   - 測試集 (未見過): {len(test_df)} 筆")
print(f"   - 測試集反諷句分佈: {test_df['sarcasm'].value_counts().to_dict()}")

# ==========================================
# 3. 載入模型與執行雙模預測
# ==========================================
print("\n🚀 正在載入模型並進行推論...")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"   Using device: {device}")

try:
    # Model A (Baseline)
    print(f"   Loading Model A from: {os.path.basename(PATH_A)}")
    model_a = SetFitModel.from_pretrained(PATH_A).to(device)
    preds_a = model_a.predict(test_df['text'].tolist())

    # Model B (Enhanced)
    print(f"   Loading Model B from: {os.path.basename(PATH_B)}")
    model_b = SetFitModel.from_pretrained(PATH_B).to(device)

    # 為 Model B 注入標籤
    def inject_sarcasm(row):
        # 對應 CSV 中的 'Yes'
        if row['sarcasm'] == 'Yes':
            return f"[Sarcasm] {row['text']}"
        else:
            return row['text']

    test_df['text_enhanced'] = test_df.apply(inject_sarcasm, axis=1)
    preds_b = model_b.predict(test_df['text_enhanced'].tolist())

except Exception as e:
    print(f"\n❌ 模型載入失敗，請檢查路徑: {e}")
    # 停止執行後續繪圖
    raise e

# 真實標籤
y_true = test_df['label_original'].tolist()
sarcasm_flags = test_df['sarcasm'].tolist()

# ==========================================
# 4. 繪製完整評估圖表
# ==========================================
print("\n📊 繪製分析圖表...")

def plot_full_analysis(y_true, preds_a, preds_b, sarcasm_flags):
    fig, axes = plt.subplots(1, 3, figsize=(22, 6))

    # 設定字型大小
    plt.rcParams.update({'font.size': 12})

    # --- 圖 1: Model A 混淆矩陣 ---
    cm_a = confusion_matrix(y_true, preds_a)
    sns.heatmap(cm_a, annot=True, fmt='d', cmap='Blues', ax=axes[0], cbar=False, annot_kws={"size": 14})
    acc_a = accuracy_score(y_true, preds_a)
    axes[0].set_title(f"Model A (Baseline)\nOverall Acc: {acc_a:.1%}", fontweight='bold')
    axes[0].set_xlabel("Predicted (0=Neg, 1=Pos)")
    axes[0].set_ylabel("True Label")

    # --- 圖 2: Model B 混淆矩陣 ---
    cm_b = confusion_matrix(y_true, preds_b)
    sns.heatmap(cm_b, annot=True, fmt='d', cmap='Greens', ax=axes[1], cbar=False, annot_kws={"size": 14})
    acc_b = accuracy_score(y_true, preds_b)
    axes[1].set_title(f"Model B (Enhanced)\nOverall Acc: {acc_b:.1%}", fontweight='bold')
    axes[1].set_xlabel("Predicted (0=Neg, 1=Pos)")
    axes[1].set_yticks([]) # 隱藏 Y 軸

    # --- 圖 3: 關鍵戰場 - 反諷 vs 一般 (Gap Analysis) ---
    mask_sarcasm = np.array(sarcasm_flags) == 'Yes'
    mask_normal = np.array(sarcasm_flags) == 'No'

    # 計算分組準確率
    scores = {
        'Normal\n(Model A)': accuracy_score(np.array(y_true)[mask_normal], np.array(preds_a)[mask_normal]),
        'Normal\n(Model B)': accuracy_score(np.array(y_true)[mask_normal], np.array(preds_b)[mask_normal]),
        'Sarcasm\n(Model A)': accuracy_score(np.array(y_true)[mask_sarcasm], np.array(preds_a)[mask_sarcasm]),
        'Sarcasm\n(Model B)': accuracy_score(np.array(y_true)[mask_sarcasm], np.array(preds_b)[mask_sarcasm]),
    }

    names = list(scores.keys())
    values = list(scores.values())
    colors = ['#aec7e8', '#98df8a', '#1f77b4', '#2ca02c'] # 淺藍, 淺綠, 深藍, 深綠

    bars = axes[2].bar(names, values, color=colors, edgecolor='black', alpha=0.8)
    axes[2].set_title("Performance Gap: Normal vs Sarcasm", fontweight='bold')
    axes[2].set_ylim(0, 1.15)

    # 標示數值
    for bar in bars:
        height = bar.get_height()
        axes[2].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                f'{height:.1%}', ha='center', va='bottom', fontweight='bold')

    plt.tight_layout()
    plt.show()

plot_full_analysis(y_true, preds_a, preds_b, sarcasm_flags)

# ==========================================
# 5. 救援成功案例展示
# ==========================================
print("\n🕵️‍♀️ [救援成功案例] Model A 誤判 -> Model B 修正 (限反諷句)：")

compare_df = test_df.copy()
compare_df['Pred_A'] = preds_a
compare_df['Pred_B'] = preds_b

# 篩選條件：A 錯, B 對, 且真的是反諷
rescued = compare_df[
    (compare_df['Pred_A'] != compare_df['label_original']) &
    (compare_df['Pred_B'] == compare_df['label_original']) &
    (compare_df['sarcasm'] == 'Yes')
]

if not rescued.empty:
    # 顯示前 5 筆，包含原始文字與注入後的文字
    display_cols = ['text', 'text_enhanced', 'label_original', 'Pred_A', 'Pred_B']
    # 為了美觀，截斷過長的文字
    pd.set_option('display.max_colwidth', 100)
    display(rescued[display_cols].head(5))
    print(f"\n共發現 {len(rescued)} 筆救援成功的反諷案例！")
else:
    print("本次測試未發現顯著的反諷救援案例。")